In [2]:
%%bash
PROJECT_ID=$(gcloud config get-value project)
DATASET="challenge5"

bq --location=US mk -d \
    --description "Aero Alerts" \
    "${PROJECT_ID}:${DATASET}" || true

Dataset 'qwiklabs-gcp-01-7c4bc0037dce:challenge5' successfully created.


In [42]:
PROJECT_ID = "qwiklabs-gcp-01-7c4bc0037dce"
REGION = "us-central1"

DATASET = "challenge5"
RAW_TABLE = "airport_raw"
AIRPORTS_TABLE = "airports"
ALERT_TABLE = "airport_alerts"
MODEL_NAME     = f"{PROJECT_ID}.{DATASET}.gemini_model"

from google.cloud import bigquery
import os
os.environ['PROJECT_ID'] = PROJECT_ID
os.environ['DATASET'] = DATASET
bq = bigquery.Client(project=PROJECT_ID)

In [5]:
!bq load \
  --source_format=CSV \
  --skip_leading_rows=1 \
  --autodetect \
  $PROJECT_ID:$DATASET.airport_raw \
  gs://labs.roitraining.com/data-to-ai-workshop/airports.csv

Waiting on bqjob_r1ff7ecdcb6ab8be2_0000019bc7493036_1 ... (2s) Current status: DONE   


In [19]:
#only extract large_airport in US to a new table
sql_table_creation = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{AIRPORTS_TABLE}` AS
SELECT *
FROM `{PROJECT_ID}.{DATASET}.{RAW_TABLE}`
WHERE type = 'large_airport' AND iso_country = 'US'
"""

job = bq.query(sql_table_creation)
job.result()
print("Created table:", f"{PROJECT_ID}.{DATASET}.{AIRPORTS_TABLE}")

Created table: qwiklabs-gcp-01-7c4bc0037dce.challenge5.airports


In [36]:
#create alert table to store forecast text, gemini response etc.
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{ALERT_TABLE}` (
  id INT64,
  ident STRING,
  name STRING,
  municipality STRING,
  iso_region STRING,
  iso_country STRING,
  iata_code STRING,
  icao_code STRING,
  latitude_deg FLOAT64,
  longitude_deg FLOAT64,

  forecast_text STRING,
  forecast_retrieved_at TIMESTAMP,

  alert_text STRING,
  alert_level STRING,
  alert_summary STRING,
  alert_generated_at TIMESTAMP
)
"""
bq.query(sql).result()
print("Created:", f"{PROJECT_ID}.{DATASET}.{ALERT_TABLE}")


Created: qwiklabs-gcp-01-7c4bc0037dce.challenge5.airport_alerts


In [37]:
#run this to get the airports table into a dataframe
airports_sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.{AIRPORTS_TABLE}`
WHERE latitude_deg IS NOT NULL AND longitude_deg IS NOT NULL
"""
airports_df = bq.query(airports_sql).to_dataframe()
print("Large airports:", len(airports_df))
airports_df.head(5)


Large airports: 71


,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
0,5388,PANC,large_airport,Ted Stevens Anchorage International Airport,61.179004,-149.992561,152,NA,US,US-AK,Anchorage,True,PANC,ANC,PANC,ANC,https://dot.alaska.gov/anc/,https://en.wikipedia.org/wiki/Ted_Stevens_Anch...,None
1,3772,KPHX,large_airport,Phoenix Sky Harbor International Airport,33.435302,-112.005905,1135,NA,US,US-AZ,Phoenix,True,KPHX,PHX,KPHX,PHX,http://phoenix.gov/skyharborairport/,https://en.wikipedia.org/wiki/Phoenix_Sky_Harb...,None
2,3892,KSMF,large_airport,Sacramento International Airport,38.695400,-121.591003,27,NA,US,US-CA,Sacramento,True,KSMF,SMF,KSMF,SMF,https://sacramento.aero/smf,https://en.wikipedia.org/wiki/Sacramento_Inter...,None
3,3752,KONT,large_airport,Ontario International Airport,34.056000,-117.600998,944,NA,US,US-CA,Ontario,True,KONT,ONT,KONT,ONT,https://www.flyontario.com/,https://en.wikipedia.org/wiki/LA/Ontario_Inter...,None
4,3883,KSJC,large_airport,Norman Y. Mineta San Jose International Airport,37.362452,-121.929188,62,NA,US,US-CA,San Jose,True,KSJC,SJC,KSJC,SJC,https://www.flysanjose.com/,https://en.wikipedia.org/wiki/San_Jose_Interna...,None


In [43]:
#gemini-external-model
import subprocess, json, re

LOCATION = "US"
CONNECTION_ID = "gemini_conn"

# Create the connection
subprocess.run([
    "bq", f"--project_id={PROJECT_ID}", "mk", "--connection",
    f"--location={LOCATION}",
    "--connection_type=CLOUD_RESOURCE",
    CONNECTION_ID
], check=False)

# Show the connection and extract the service account
out = subprocess.check_output([
    "bq", f"--project_id={PROJECT_ID}", "show", "--format=prettyjson",
    "--connection", f"{PROJECT_ID}.{LOCATION}.{CONNECTION_ID}"
], text=True)

conn = json.loads(out)
sa = conn["cloudResource"]["serviceAccountId"]
print("Connection service account:", sa)

# Grant Vertex AI User role to the connection service account
subprocess.check_call([
    "gcloud", "projects", "add-iam-policy-binding", PROJECT_ID,
    "--member", f"serviceAccount:{sa}",
    "--role", "roles/aiplatform.user"
])

print("Granted roles/aiplatform.user to:", sa)

#gemini model
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

CONN_REF = f"{LOCATION}.{CONNECTION_ID}"

create_model_sql = f"""
CREATE OR REPLACE MODEL `{MODEL_NAME}`
REMOTE WITH CONNECTION `{CONN_REF}`
OPTIONS (
  ENDPOINT = 'gemini-2.5-flash'
)
"""
client.query(create_model_sql).result()
print("Created remote model:", MODEL_NAME)

Connection service account: bqcx-603410550-olb7@gcp-sa-bigquery-condel.iam.gserviceaccount.com
Granted roles/aiplatform.user to: bqcx-603410550-olb7@gcp-sa-bigquery-condel.iam.gserviceaccount.com
Created remote model: qwiklabs-gcp-01-7c4bc0037dce.challenge5.gemini_model


In [44]:
import requests, time
from datetime import datetime, timezone

HEADERS = {
    "User-Agent": "qwiklabs-aero-alerts",
    "Accept": "application/geo+json"
}

def nws_forecast_text(lat: float, lon: float, periods_n: int = 4) -> str:
    points_url = f"https://api.weather.gov/points/{lat},{lon}"
    r = requests.get(points_url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    forecast_url = r.json()["properties"]["forecast"]

    r2 = requests.get(forecast_url, headers=HEADERS, timeout=30)
    r2.raise_for_status()
    periods = r2.json()["properties"]["periods"]

    lines = []
    for p in periods[:periods_n]:
        lines.append(f"{p['name']}: {p['detailedForecast']}")
    return "\n".join(lines)

# Reset the table content each run (keeps it clean)
bq.query(f"TRUNCATE TABLE `{PROJECT_ID}.{DATASET}.{ALERT_TABLE}`").result()

now_ts = datetime.now(timezone.utc).isoformat()
rows = []

for _, a in airports_df.iterrows():
    lat = float(a["latitude_deg"])
    lon = float(a["longitude_deg"])

    try:
        forecast = nws_forecast_text(lat, lon, periods_n=4)
    except Exception as e:
        forecast = f"ERROR retrieving forecast: {e}"

    rows.append({
        "id": int(a["id"]),
        "ident": a.get("ident"),
        "name": a.get("name"),
        "municipality": a.get("municipality"),
        "iso_region": a.get("iso_region"),
        "iso_country": a.get("iso_country"),
        "iata_code": a.get("iata_code"),
        "icao_code": a.get("icao_code"),
        "latitude_deg": lat,
        "longitude_deg": lon,
        "forecast_text": forecast,
        "forecast_retrieved_at": now_ts,
        "alert_text": None,
        "alert_level": None,
        "alert_summary": None,
        "alert_generated_at": None,
    })

    time.sleep(0.5)

errors = bq.insert_rows_json(f"{PROJECT_ID}.{DATASET}.{ALERT_TABLE}", rows)
if errors:
    raise RuntimeError(errors)

print("Populated forecasts into:", ALERT_TABLE, "rows:", len(rows))

Populated forecasts into: airport_alerts rows: 71


In [31]:
# unit test on if we could get responses from the weather_api
row = airports_df.iloc[0]
print(row["name"])
print(nws_forecast_text(row["latitude_deg"], row["longitude_deg"]))

Ted Stevens Anchorage International Airport
Today: Snow and freezing rain before 9am, then rain and snow between 9am and noon, then a chance of snow showers between noon and 3pm. Partly sunny. High near 34, with temperatures falling to around 25 in the afternoon. South wind 0 to 35 mph, with gusts as high as 55 mph. Chance of precipitation is 80%. New snow accumulation of less than half an inch possible.
Tonight: Partly cloudy, with a low around 17. East wind 0 to 10 mph.
Saturday: Snow likely after noon. Mostly cloudy, with a high near 29. North wind 10 to 15 mph. Chance of precipitation is 60%. New snow accumulation of less than one inch possible.
Saturday Night: A slight chance of snow before 9pm, then a slight chance of freezing rain. Mostly cloudy, with a low around 24. North wind around 10 mph.


In [51]:
sql_gemini_min = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{ALERT_TABLE}` AS
WITH gen AS (
  SELECT
    x.*,
    JSON_VALUE(x.ml_generate_text_result, '$.candidates[0].content.parts[0].text') AS _model_text
  FROM ML.GENERATE_TEXT(
    MODEL `{MODEL_NAME}`,
    (
      SELECT
        CONCAT(
          'You are an aviation operations assistant. ',
          'Output STRICT JSON only. Do NOT wrap in ``` fences. ',
          'Keys: alert_level (GREEN/YELLOW/RED), alert_summary (2-4 sentences, actionable), key_hazards (array). ',
          'Airport name: ', COALESCE(name, ''), '. ',
          'Forecast: ', COALESCE(forecast_text, '')
        ) AS prompt,
        t.*
      FROM `{PROJECT_ID}.{DATASET}.{ALERT_TABLE}` t
      WHERE forecast_text IS NOT NULL
    ),
    STRUCT(0.2 AS temperature, 1024 AS max_output_tokens)
  ) AS x
),
final AS (
  SELECT
    * EXCEPT(
      prompt, ml_generate_text_result, ml_generate_text_status,
      raw_text, alert_json, alert_json_clean,
      alert_text, alert_level, alert_summary, alert_generated_at
    ),

    TRIM(
      REGEXP_REPLACE(
        REGEXP_REPLACE(_model_text, r'(?is)```json', ''),
        r'(?is)```', ''
      )
    ) AS _cleaned_json
  FROM gen
)
SELECT
  * EXCEPT(_cleaned_json),
  _cleaned_json AS alert_text,
  JSON_VALUE(_cleaned_json, '$.alert_level') AS alert_level,
  JSON_VALUE(_cleaned_json, '$.alert_summary') AS alert_summary,
  CURRENT_TIMESTAMP() AS alert_generated_at
FROM final
"""
bq.query(sql_gemini_min).result()
print("Rewritten table with minimal columns:", f"{PROJECT_ID}.{DATASET}.{ALERT_TABLE}")


Rewritten table with minimal columns: qwiklabs-gcp-01-7c4bc0037dce.challenge5.airport_alerts


In [49]:
sql_gemini = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.{ALERT_TABLE}` AS
WITH gen AS (
  SELECT
    x.*,
    JSON_VALUE(x.ml_generate_text_result, '$.candidates[0].content.parts[0].text') AS raw_text
  FROM ML.GENERATE_TEXT(
    MODEL `{MODEL_NAME}`,
    (
      SELECT
        CONCAT(
          'You are an aviation operations assistant. ',
          'Output STRICT JSON only. Do NOT wrap in ``` fences. ',
          'Keys: alert_level (GREEN/YELLOW/RED), alert_summary (2-4 sentences), key_hazards (array). ',
          'Airport name: ', COALESCE(name, ''), '. ',
          'Forecast: ', COALESCE(forecast_text, '')
        ) AS prompt,
        t.*
      FROM `{PROJECT_ID}.{DATASET}.{ALERT_TABLE}` t
      WHERE forecast_text IS NOT NULL
    ),
    STRUCT(0.2 AS temperature, 2000 AS max_output_tokens)
  ) AS x
),
clean AS (
  SELECT
    * EXCEPT(raw_text),
    raw_text,
    TRIM(
      REGEXP_REPLACE(
        REGEXP_REPLACE(raw_text, r'(?is)```json', ''),
        r'(?is)```', ''
      )
    ) AS alert_json_clean
  FROM gen
)
SELECT
  * EXCEPT(prompt, ml_generate_text_result, ml_generate_text_status,
           alert_text, alert_level, alert_summary, alert_generated_at),

  alert_json_clean AS alert_text,
  JSON_VALUE(alert_json_clean, '$.alert_level') AS alert_level,
  JSON_VALUE(alert_json_clean, '$.alert_summary') AS alert_summary,
  CURRENT_TIMESTAMP() AS alert_generated_at
FROM clean
"""
bq.query(sql_gemini).result()
print("Regenerated + parsed alerts into:", f"{PROJECT_ID}.{DATASET}.{ALERT_TABLE}")


Regenerated + parsed alerts into: qwiklabs-gcp-01-7c4bc0037dce.challenge5.airport_alerts
